In [1]:
# !pip install lightgbm

## Importing Dependencies

In [2]:
# to store data
import pandas as pd
#to do linear algebra 
import numpy as np
from numpy import pi

# for plotting
from matplotlib.colors import rgb2hex
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt

#for nicer plots 
import seaborn as sns

# fro interacive plots 
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected= True)

# to get new dtatypes and functions
from collections import Counter
from cycler import cycler

# to investigate distributions
from scipy.stats import skew, norm, probplot
from scipy.optimize import curve_fit

# To Build models 

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#to bgm light 
from lightgbm import LGBMClassifier

# For time
from time import time

## Load Data

In [3]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# to add new coloumn
train_data['Data'] = 'Train'
test_data['Data'] = 'Test'
# this will help in connctenated data frame
# Concatenatting
combined_data = pd.concat([train_data, test_data], axis=0).reset_index(drop = True)
combined_data['subject'] = '#' + combined_data['subject'].astype(str)

# Lable 
lable = combined_data.pop('Activity')

In [4]:
train_data.shape

(7352, 564)

In [5]:
test_data.shape

(2947, 564)

In [6]:
combined_data.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Data
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,#1,Train
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,#1,Train
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,#1,Train
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,#1,Train
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,#1,Train


In [7]:
# to show nul values 
combined_data.isna().sum().sum()

0

In [8]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10299 entries, 0 to 10298
Columns: 563 entries, tBodyAcc-mean()-X to Data
dtypes: float64(561), object(2)
memory usage: 44.2+ MB


In [9]:
combined_data.info

<bound method DataFrame.info of        tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  \
0               0.288585          -0.020294          -0.132905   
1               0.278419          -0.016411          -0.123520   
2               0.279653          -0.019467          -0.113462   
3               0.279174          -0.026201          -0.123283   
4               0.276629          -0.016570          -0.115362   
...                  ...                ...                ...   
10294           0.310155          -0.053391          -0.099109   
10295           0.363385          -0.039214          -0.105915   
10296           0.349966           0.030077          -0.115788   
10297           0.237594           0.018467          -0.096499   
10298           0.153627          -0.018437          -0.137018   

       tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  \
0             -0.995279         -0.983111         -0.913526         -0.995112   
1            

## Distributing Lables

In [10]:
lable_counts = lable.value_counts()
lable_counts

LAYING                1944
STANDING              1906
SITTING               1777
WALKING               1722
WALKING_UPSTAIRS      1544
WALKING_DOWNSTAIRS    1406
Name: Activity, dtype: int64

In [11]:
n = lable_counts.shape[0]
n

6

In [12]:
# Get Colors 
colormap = get_cmap("viridis")
colors = [rgb2hex(colormap(col)) for col in np.arange(0,1.01, 1/(n-1))]

data = go.Bar( x=lable_counts.index,
            y = lable_counts,
            marker = dict(color = colors))

layout = go.Layout( title = "Lable Activity Distribution",
                  xaxis = dict(title= "Activity"),
                  yaxis = dict(title = "Counts"))

fig = go.Figure( data=[data], layout=layout)
iplot(fig)
                

## SVM for cllassification purposes

In [13]:
import numpy as np 
import pandas as pd 
import seaborn as sns
# import paylab as pl
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, GridSearchCV
import os 

os.listdir("../UCI Dataset")

['.ipynb_checkpoints',
 'HumanActivityRecUsingSVM.ipynb',
 'test.csv',
 'train.csv']

## Load Train and Test Data

In [14]:
train_data_frame = shuffle(pd.read_csv("train.csv"))
test_data_frame = shuffle(pd.read_csv("test.csv"))


In [15]:
# to show nul values 
print("Any missing value in train data frame:",train_data_frame.isna().sum().sum())
# to show nul values 
print("Any missing value in test data frame:",test_data_frame.isna().sum().sum())

Any missing value in train data frame: 0
Any missing value in test data frame: 0


In [16]:
# Frequency Distribution of the outcome
train_outcome = pd.crosstab(index=train_data_frame['Activity'],
                           columns='Counts')
train_outcome

col_0,Counts
Activity,
LAYING,1407
SITTING,1286
STANDING,1374
WALKING,1226
WALKING_DOWNSTAIRS,986
WALKING_UPSTAIRS,1073


## Normalizing the predictor for SVM

In [17]:
# Separating predictors and outcome values from train set and test set

x_train = pd.DataFrame(train_data_frame.drop(['Activity','subject' ], axis=1))
y_train_lable = train_data_frame.Activity.values.astype(object)
x_test = pd.DataFrame(test_data_frame.drop(['Activity','subject' ], axis=1))
y_test_lable = test_data_frame.Activity.values.astype(object)


print("Diemention of x_train:",x_train.shape)
print("Diemention of x_test:",x_test.shape)


Diemention of x_train: (7352, 561)
Diemention of x_test: (2947, 561)


In [19]:
# Transferring non_numerical lables into numiracls
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

## Encoding Train lables
encoder.fit(y_train_lable)
train_lables=encoder.transform(y_train_lable)

## Encoding Test lables
encoder.fit(y_test_lable)
test_lables=encoder.transform(y_test_lable)

# Getting number of continuse and categral functions in the training set
num_colomns = x_train._get_numeric_data().columns
num_colomns.size

561

In [24]:
num_of_predictors = list(x_train.columns.values)

# Scalling the train and test features test 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_scalled = scaler.fit_transform(x_train)
x_test_scalled = scaler.transform(x_test)


## Hypermeter tunning using Grid Search and cross validation 

In [25]:
# Libraries to build groupe model : Random Forest Classifier 
from sklearn.model_selection import ParameterGrid
params_grid =[{'kernel':['rbf'],'gamma':[1e-3,1e-4],'C':[1,10,100,1000]},
             {
                 'kernel':['linear'], 'C':[1,10,100,1000]
             }]

## Training SVM Model using Radial Kernal 

In [26]:
svm_model = GridSearchCV(SVC(), params_grid, cv= 5)
svm_model.fit(x_train_scalled, y_train_lable)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

## Confusion Matrix and accuracy Score

In [27]:
# View Acuracy score
print("Accuracy Score of Training Data", svm_model.best_score_,"\n")

# View the best paramters using grid search
print("Best Scores of C:", svm_model.best_estimator_.C,"\n")
print("Best Scores Kernal:", svm_model.best_estimator_.kernel,"\n")
print("Best Scores Gamma:", svm_model.best_estimator_.gamma,"\n")

Accuracy Score of Training Data 0.9880303555820698 

Best Scores of C: 100 

Best Scores Kernal: rbf 

Best Scores Gamma: 0.001 



In [28]:
# Final Model 

final_model = svm_model.best_estimator_
y_predict = final_model.predict(x_test_scalled)
# y_pred_lable = list(encoder.transform(y_predict))
y_predict


array(['STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', ...,
       'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS'],
      dtype=object)

In [29]:
## Making confusion Matrix
print(confusion_matrix(y_test_lable,y_predict))
print("\n")
print(classification_report(y_test_lable, y_predict))
print("\n")
print("Training Set SCore for SVM:", final_model.score(x_train_scalled,y_train_lable))
print("\n")
print("Testingt Score for SVM:", final_model.score(x_test_scalled,y_test_lable))


[[537   0   0   0   0   0]
 [  3 440  47   0   0   1]
 [  0  13 519   0   0   0]
 [  0   0   0 486   4   6]
 [  0   0   0   6 389  25]
 [  0   0   0  15   2 454]]


                    precision    recall  f1-score   support

            LAYING       0.99      1.00      1.00       537
           SITTING       0.97      0.90      0.93       491
          STANDING       0.92      0.98      0.95       532
           WALKING       0.96      0.98      0.97       496
WALKING_DOWNSTAIRS       0.98      0.93      0.95       420
  WALKING_UPSTAIRS       0.93      0.96      0.95       471

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



Training Set SCore for SVM: 1.0


Testingt Score for SVM: 0.9586019681031558
